In [2]:
import pandas as pd
import sys
sys.path.append('/home/brayan/mnsun/')
from utils import check_dir
import os

## Create Scrip 

In [11]:
actual_date   = "20181101" #"".join(str(os.environ["date_processing"]).split("-"))
base_line     = "wget https://noaa-cfs-pds.s3.amazonaws.com/cfs.{ACTUAL_DATE}/{INIT}/6hrly_grib_{ENS}/ocnf{DATE_FORECAST}{INIT_FORE}.{ENS}.{ACTUAL_DATE}{INIT}.grb2"
OUTPUT_DIR    = "/home/brayan/DATA/AWS/" #str(os.environ["output"])
check_dir( OUTPUT_DIR, name=actual_date)
init_hour     = {"00":"06", "06":"12", "12":"18", "18":"00"}
date_forecast = pd.date_range(actual_date, freq="D", periods=45+1)

In [12]:
date_forecast

DatetimeIndex(['2018-11-01', '2018-11-02', '2018-11-03', '2018-11-04',
               '2018-11-05', '2018-11-06', '2018-11-07', '2018-11-08',
               '2018-11-09', '2018-11-10', '2018-11-11', '2018-11-12',
               '2018-11-13', '2018-11-14', '2018-11-15', '2018-11-16',
               '2018-11-17', '2018-11-18', '2018-11-19', '2018-11-20',
               '2018-11-21', '2018-11-22', '2018-11-23', '2018-11-24',
               '2018-11-25', '2018-11-26', '2018-11-27', '2018-11-28',
               '2018-11-29', '2018-11-30', '2018-12-01', '2018-12-02',
               '2018-12-03', '2018-12-04', '2018-12-05', '2018-12-06',
               '2018-12-07', '2018-12-08', '2018-12-09', '2018-12-10',
               '2018-12-11', '2018-12-12', '2018-12-13', '2018-12-14',
               '2018-12-15', '2018-12-16'],
              dtype='datetime64[ns]', freq='D')

In [13]:
for ens in ["01"]: #,"02", "03", "04"
    check_dir( OUTPUT_DIR+f"/{actual_date}/", name=ens)
    with open(f"{OUTPUT_DIR}" + f"/{actual_date}/{ens}/" + actual_date + "_download.txt", "w") as f:
        for hour in ["00", "06", "12", "18"]:
            for t in range(45):
                if t == 0:
                    f.write(base_line.format(ACTUAL_DATE = actual_date, INIT = hour, ENS = ens, INIT_FORE = init_hour[hour],
                                             DATE_FORECAST = ''.join(str(date_forecast[t+1])[:10].split("-")) if hour == "18" else ''.join(str(date_forecast[t])[:10].split("-")) )+ "\n")
                else:
                    f.write(base_line.format(ACTUAL_DATE = actual_date, INIT = hour, ENS = ens, INIT_FORE = hour,
                             DATE_FORECAST = ''.join(str(date_forecast[t+1])[:10].split("-")) if hour == "18" else ''.join(str(date_forecast[t])[:10].split("-")) )+ "\n")

# Create day data

In [6]:
import xarray as xr
import warnings
import numpy as np
import pandas as pd
import xesmf as xe
import sys
import os
sys.path.append('/home/brayan/mnsun/')
from utils import check_dir
warnings.filterwarnings('ignore')

In [7]:
# Dictorionary acumulation
day_year = {}
acum     = 0
day_year["01"] = acum
for contador, dm in enumerate([31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30]):
    acum += dm
    if contador < 8:
        day_year[f"0{contador+2}"] = acum
    else: 
        day_year[f"{contador+2}"] = acum

In [16]:
# Start code
actual_date = "20181031" # "".join(str(os.environ["date_processing"]).split("-"))
OUTPUT_DIR  = "/home/brayan/DATA/AWS/" #str(os.environ["output"])

In [14]:
var = "uflx"

In [15]:
ens = "01"

In [18]:
DATA = xr.open_mfdataset(OUTPUT_DIR+actual_date+f"/{ens}/*{actual_date}00.grb2", 
                                 engine="cfgrib", concat_dim = 'time', combine='nested').sel(latitude=slice(30,-30),
                                                                           longitude=slice(90,298))

DatasetBuildError: multiple values for unique key, try re-open the file with one of:
    filter_by_keys={'typeOfLevel': 'depthBelowSea'}
    filter_by_keys={'typeOfLevel': 'surface'}
    filter_by_keys={'typeOfLevel': 'unknown'}
    filter_by_keys={'typeOfLevel': 'oceanIsotherm'}